In [ ]:
# Install necessary libraries
!pip install deepface opencv-python-headless mtcnn pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.3/128.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.8 MB/s eta 0:00:00


In [ ]:
!pip install deepface mtcnn openpyxl

import cv2
import numpy as np
import pandas as pd
from mtcnn import MTCNN
from deepface import DeepFace
from collections import defaultdict
import os

# Paths
video_path = "/content/drive/MyDrive/clarifai/input/concentrated students video.mp4"
output_video_path = "/content/drive/MyDrive/clarifai/output/output_resnet_concentrated.mp4"
excel_path = "/content/drive/MyDrive/clarifai/output/detections_resnet_concentrated.xlsx"

# Initialize
detector = MTCNN()
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (w, h))

# Tracking: store last known face centers to assign IDs
next_id = 1
face_centers = {}
tolerance = 50  # pixel distance to match same student

# Data storage
data = []

def assign_id(center):
    global next_id
    for fid, prev_center in face_centers.items():
        if np.linalg.norm(np.array(center) - np.array(prev_center)) < tolerance:
            face_centers[fid] = center
            return fid
    face_centers[next_id] = center
    next_id += 1
    return next_id - 1

def map_emotion(emotion):
    return emotion

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    faces = detector.detect_faces(rgb)

    for face in faces:
        x, y, w_, h_ = face['box']
        x, y = max(0, x), max(0, y)
        face_crop = rgb[y:y+h_, x:x+w_]

        try:
            # DeepFace emotion prediction
            result = DeepFace.analyze(face_crop, actions=['emotion'], enforce_detection=False)
            emotion = result[0]['dominant_emotion']
            precision = result[0]['emotion'][emotion]
        except:
            emotion = "unknown"
            precision = 0

        engagement = map_emotion(emotion)
        center = (x + w_//2, y + h_//2)
        fid = assign_id(center)

        # Draw
        color = (0, 255, 0) if engagement == 'Engaged' else (0, 0, 255)
        cv2.rectangle(frame, (x, y), (x + w_, y + h_), color, 2)
        cv2.putText(frame, f"ID:{fid} {engagement}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

        # Log data
        timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
        data.append([timestamp, fid, engagement, emotion, precision])

    out.write(frame)

cap.release()
out.release()

# Save Excel
df = pd.DataFrame(data, columns=['Time (s)', 'Student ID', 'State', 'Emotion', 'Precision'])
df.to_excel(excel_path, index=False)

print(f"Video saved to: {output_video_path}")
print(f"Excel saved to: {excel_path}")
